# Project - 2


`Libraries`


In [3]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import csv
import numpy as np

`Plotting Functions`


1. Euler Explicit scheme


2.  Euler Implicit scheme


3. Crank-Nicolson scheme
